In [3]:
!pip install -r requirements.txt

  Using cached numpy-2.2.2-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached transformers-4.48.1-py3-none-any.whl.metadata (44 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 35.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 31.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.7 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 29.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.

In [2]:
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from importlib import import_module
import os

from src.generate import gen
from src.prompt_tones import get_change_tone_prompt

/Users/gbndict/.local/share/mise/installs/python/3.12.3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dynaconf import Dynaconf

settings = Dynaconf(settings_files=["settings.toml"], 
                    env="qwen2-1-5B", # phi3-8B
                    environments=True)

In [4]:
tone = "SHORTEN"

In [8]:
if os.path.isdir(settings.model):
    tokenizer = AutoTokenizer.from_pretrained(settings.model)
    model = AutoModelForCausalLM.from_pretrained(settings.model)
    model.eval().cuda()

In [ ]:
input_text = "Here's some sample text to change."

messages = get_change_tone_prompt(tone, input_text, tokenizer)
print(messages)

In [103]:
module_name = f"data.tones.{tone.lower()}_this"

module = import_module(module_name)

object_name = tone.lower() + "_this"
queries = getattr(module, object_name)

In [104]:
queries[:5]

["The quick brown fox jumps over the lazy dog. It's a well-known pangram, a sentence that contains all the letters of the alphabet. This phrase has been used to test typewriters, computer keyboards, and other devices that involve text input.",
 "In the realm of software development, Python has emerged as a powerful and versatile programming language. Its simplicity, readability, and vast ecosystem of libraries have made it a go-to choice for developers across various domains, from web development and data analysis to machine learning and automation. Python's popularity continues to soar, thanks to its user-friendly syntax and cross-platform compatibility.",
 'The majestic Himalayas, a towering mountain range that stretches across several countries in Asia, have captivated the imagination of adventurers and nature enthusiasts for centuries. These snow-capped peaks, some of the highest in the world, offer breathtaking vistas, challenging trekking routes, and a unique opportunity to immer

In [105]:
predictions = [
    gen(get_change_tone_prompt(tone, e, tokenizer), tokenizer, model) 
    for e in tqdm(queries)
]

100%|██████████| 98/98 [01:04<00:00,  1.51it/s]


In [106]:
predictions

['It\'s a pangram! The phrase "The quick brown fox jumps over the lazy dog" is famous for its complete use of the English alphabet.',
 "Python is a popular programming language known for its simplicity, versatility, and extensive library ecosystem. It's widely used in areas like web development, data analysis, machine learning, and automation due to its readability and compatibility across platforms.",
 'Yes, the Himalayas are truly awe-inspiring. The trekking routes there are both thrilling and rewarding. It’s an incredible experience to immerse yourself in such a diverse culture too!',
 'Artificial Intelligence has transformed industries and daily life, impacting everything from virtual assistants to self-driving cars. AI algorithms are transforming how we interact with machines and process data.',
 'Sustainability is gaining importance globally due to environmental concerns. It focuses on meeting current needs without compromising the ability of future generations to meet their own 